In [1]:
import matplotlib.pyplot as plt
import os
import torch 
from torch.utils.data import DataLoader
import json
import sys
import torch

sys.path.append("src")
os.chdir('/home/george-vengrovski/Documents/projects/tweety_bert_paper')

from data_class import CollateFunction
from utils import load_model

weights_path = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/experiments/TweetyBERT-MSE-Mask-Before-50-mask-alpha-1/saved_weights/model_step_6400.pth"
config_path = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/experiments/TweetyBERT-MSE-Mask-Before-50-mask-alpha-1/config.json"

tweety_bert_model = load_model(config_path, weights_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


KeyError: 'num_freq_bins'

## Data Class

In [9]:
from torch.utils.data import DataLoader
from data_class import SongDataSet_Image

train_dir = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb3_train"
test_dir = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb3_test"

train_dataset = SongDataSet_Image(train_dir, num_classes=196)
test_dataset = SongDataSet_Image(test_dir, num_classes=196)

collate_fn = CollateFunction(segment_length=1000)  # Adjust the segment length if needed

train_loader = DataLoader(train_dataset, batch_size=48, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=48, shuffle=True, collate_fn=collate_fn)

## Define Linear Classifier and Train

In [10]:
from linear_probe import LinearProbeModel, LinearProbeTrainer

classifier_model = LinearProbeModel(num_classes=21, model_type="neural_net", model=tweety_bert_model, freeze_layers=False, layer_num=2, layer_id="attention_output", classifier_dims=196)
classifier_model = classifier_model.to(device)

In [12]:
trainer = LinearProbeTrainer(model=classifier_model, train_loader=train_loader, test_loader=test_loader, device=device, lr=3e-4, plotting=True, batches_per_eval=10, desired_total_batches=100, patience=4)
trainer.train()

Batch 10: FER = 49.30%, Train Loss = 6.2491, Val Loss = 4.9879
Batch 20: FER = 23.11%, Train Loss = 3.0641, Val Loss = 2.6108
Batch 30: FER = 14.07%, Train Loss = 1.7632, Val Loss = 1.4944
Batch 40: FER = 8.23%, Train Loss = 0.9761, Val Loss = 0.8982
Batch 50: FER = 7.65%, Train Loss = 0.4146, Val Loss = 0.4678
Batch 60: FER = 6.46%, Train Loss = 0.3548, Val Loss = 0.3687
Batch 70: FER = 5.91%, Train Loss = 0.4697, Val Loss = 0.2648
Batch 80: FER = 5.88%, Train Loss = 0.2781, Val Loss = 0.2467
Batch 90: FER = 5.89%, Train Loss = 0.2046, Val Loss = 0.2413
Batch 100: FER = 4.98%, Train Loss = 0.1679, Val Loss = 0.1865
Batch 110: FER = 4.58%, Train Loss = 0.2109, Val Loss = 0.1724
Batch 120: FER = 4.88%, Train Loss = 0.1643, Val Loss = 0.1813
Batch 130: FER = 5.81%, Train Loss = 0.1621, Val Loss = 0.2253
Batch 140: FER = 5.74%, Train Loss = 0.1445, Val Loss = 0.2085
Batch 150: FER = 4.84%, Train Loss = 0.1713, Val Loss = 0.1774
Batch 160: FER = 4.43%, Train Loss = 0.1671, Val Loss = 0.156

KeyboardInterrupt: 

## Analyze

In [1]:
from linear_probe import ModelEvaluator


# Initialize the ModelEvaluator with the filter_unseen_classes feature
evaluator = ModelEvaluator(model=classifier_model, 
                           test_loader=test_loader, 
                           num_classes=21,  # Assuming there are 21 possible classes
                           device='cuda:0',  # Use CUDA if available
                           use_tqdm=True,  # Enable a progress bar for the evaluation
                           filter_unseen_classes=True,  # Enable filtering based on training set classes
                           train_dir=train_dir)  # Path to the training dataset directory

# Perform model validation with multiple passes
class_frame_error_rates, total_frame_error_rate = evaluator.validate_model_multiple_passes(num_passes=1, max_batches=1250)

# Save the results to a specified directory, for example 'evaluation_results'
evaluator.save_results(class_frame_error_rates, total_frame_error_rate, '')


ModuleNotFoundError: No module named 'linear_probe'